In [1]:
# bibliotecas
import pandas as pd
from random import seed
from random import random

In [2]:
# lendo os arquivos de entrada
dataset = open("dataset.txt","r+")
initial_weights = open("initial_weights.txt","r+")
network = open("network.txt","r+")

#### Rede neural composta por um número ajustável de neurônios e camadas e treinada via backpropagation

In [3]:
def initialize_network(n_inputs, n_hidden, n_outputs, hidden_weights, output_weights):
    network = list()
    
    print('n_inputs', n_inputs)
    print('n_hidden', n_hidden)
    print('n_outputs', n_outputs)
    print(hidden_weights)
    
    i = 1
    print([random() for i in range(n_inputs)])
    
    hidden_layer = [{'weights':[random() for i in range(n_inputs)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[0.7, 0.5, 0.6]}]
    network.append(output_layer)
    return network


def_network = list()
for line in network:
    def_network.append(line.rstrip())

def read_weights(file, n_hidden, n_output):
    """ n_hidden is number of weights to neurons """
    """ n_output is number of weights to output """
    hidden_weights = list()
    output_weights = list()
    lines = file.readlines()
    i = 0
    while i < n_hidden:
        hidden_weights.append([line.split(', ') for line in lines[i].rstrip().split('; ')])
        i += 1
    while i < n_hidden + n_output:
        output_weights.append([line.split(', ') for line in lines[i].rstrip().split('; ')])
        i += 1
    return hidden_weights, output_weights
        
    
hidden_weights, output_weights = read_weights(initial_weights, 1, 1)

nn = initialize_network(int(def_network[1]), int(def_network[2]), int(def_network[3]), hidden_weights, output_weights)
print(nn[0])
print(nn[1])

n_inputs 2
n_hidden 3
n_outputs 1
[[['0.4', '0.1'], ['0.3', '0.2']]]
[0.3844171880676751, 0.23063730478019573]
[{'weights': [0.7116484156678035, 0.15857008773583325]}, {'weights': [0.538954363667775, 0.5235963195247969]}, {'weights': [0.3005311181414514, 0.5977497610241934]}]
[{'weights': [0.7, 0.5, 0.6]}]


#### Funcionalidade que permita, via linha de comando, informar a sua implementação a estrutura de uma rede de teste (i.e., estrutura de camadas/neurônios, pesos iniciais, e fator de regularização), e um conjunto de treinamento, e que retorne o gradiente calculado para cada peso

#### Funcionalidade que permita, via linha de comando, efetuar a verificação numérica do gradiente, a fim de checar a corretude da implementação de cada grupo

#### Um mecanismo para normalização das features/dados de treinamentos

#### Mecanismo para uso de regularização